## Object Detection using YOLO v8

- Created for the **Integrated Robotics and Computer Vision Workshop** for **Shaastra 2024**

- This notebook is a part of the **Computer Vision** module

- Workshop Presenters:
    - Raghav Jangid [ME20B143]
    - Swapnil Mehta [ME20B183]
    - Rahul [ME20B145]

### Importing Libraries

In [56]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image
from skimage import morphology
from skimage.feature import corner_harris,corner_peaks
import matplotlib.pyplot as plt

In [2]:
# Function to display an image -

def show_image(img,title):
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Training the YOLO v8 model to detect boxes

In [10]:
# Importing ultralytics and roboflow for YOLO v8 mode and datasets -

# Ultralytics for using YOLO model
#!pip install ultralytics      

# Roboflow for downloading training datasets
#!pip install roboflow         

from roboflow import Roboflow
rf = Roboflow(api_key="X9AIXoNO9Q8uxUXDCPXY")
project = rf.workspace("directed-technologies").project("cardboardbox-detection")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.228, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Cardboardbox-detection-1 in yolov8:: 100%|██████████| 368/368 [00:00<00:00, 882.84it/s]


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True

> **Note:** The training process is not included in this notebook as it would take a lot of time to train on the dataset.
>
> Therefore, we will be using a pre-trained model for the detection task.

### Pretrained YOLO model

In [3]:
from roboflow import Roboflow

rf = Roboflow(api_key="X9AIXoNO9Q8uxUXDCPXY")
project = rf.workspace().project("cardboardbox-detection")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [196]:
boxes_img = cv2.imread("Boxes.jpg")

show_image(boxes_img,"Boxes")   # Displaying the image

In [198]:
confidence = 40
overlap = 30
model.predict("Boxes.jpg",confidence=80,overlap=30).save("prediction_boxes.jpg")

> The above model does the prediction on the image and returns the bounding boxes and the class of the object detected which are above the **confidence imterval**.

In [199]:
detected_boxes_img = cv2.imread("prediction_boxes.jpg")

show_image(detected_boxes_img,"Boxes Detected")  # Displaying the image

In [174]:
bounding_boxes = model.predict("Boxes_3.jpg",confidence=80,overlap=30).json()["predictions"]

In [176]:
detected_boxes = []
for i in range(len(bounding_boxes)) :
    x = int(bounding_boxes[i]['x'])
    y = int(bounding_boxes[i]['y'])
    height = int(bounding_boxes[i]['height'])
    width = int(bounding_boxes[i]['width'])
    detected_boxes.append(boxes_img[y-height//2-20:y+height//2+20,x-width//2-20:x+width//2+20,:])

In [186]:
# To avoid the problem of lighting conditions, we use a range of colors instead of a single color
img = detected_boxes[0]

#box_color_range = [(60,110,150),(130,180,220)]  # Color range of the boxes
box_color_range = [(60,110,150),(130,180,240)]
lower = np.array(box_color_range[0],dtype=np.uint8)  # Lower bound of the color range
upper = np.array(box_color_range[1],dtype=np.uint8)  # Upper bound of the color range

box_mask = cv2.inRange(img,lower,upper)  # Masking the image

box_masked_img = cv2.bitwise_and(img,img,mask=box_mask)  # Masked image
show_image(box_mask,'masked image')

In [189]:
side = 10

# Create a circular kernel using skimage.morphology.disk
#circular_kernel = morphology.disk(side)
circular_kernel = np.ones((side,side),np.uint8)
# Perform morphological dilation using binary_dilation
dilated_image = cv2.dilate(box_mask,circular_kernel,iterations=1)
#dilated_image = dilated_image.astype(np.uint)
#show_image(dilated_image,'dialated')


In [ ]:
blurred_image = cv2.GaussianBlur(dilated_image,(side,side),0)

In [193]:
measure_image = corner_harris(blurred_image)
#coords = corner_peaks(measure_image, min_distance=5)
show_image(measure_image,'measure')


In [107]:
boxes_img_gray = cv2.cvtColor(box_masked_img,cv2.COLOR_BGR2GRAY)  # Converting the image to grayscale

boxes_canny = cv2.Canny(box_mask,90,100)  # Detecting edges in the image

show_image(boxes_canny,'canny edge')